The class need the following functions:
- solve for theta
- solve for x
- form M, N
- Calculate x, lambda as function of theta
- Calculate boundaries of the region

**Trial and error shows that tolerance of identification of active set is of crucial importance!!!!!! If wrong (e.g. got the constraint labelled as active when it is not) the results are very wrong!!!!!**

In [163]:
import pyomo.environ as pmo
import numpy as np

class GenericSolver:
    
    def __init__(self, A, b, Q, m):
        # get no of var and constraints
        self.x_card = np.shape(A)[1]
        self.c_card = np.shape(A)[0]
        
        # transform A from matrix to dict
        A_init = {}
        for i in range(self.c_card):
            for j in range(self.x_card):
                A_init[(i+1, j+1)] = A[i, j]
        
        # transform b from vector to dict
        b_init = {}
        for i in range(self.c_card):
            b_init[i+1] = b[i]
            
        # transform Q from vector to dict
        Q_init = {}
        for i in range(self.x_card):
            for j in range(self.x_card):
                Q_init[(i+1, j+1)] = Q[i, j]
        
        # transform m from vector to dict
        m_init = {}
        for i in range(self.x_card):
            m_init[i+1] = m[i]
        
        # define pyomo model
        self.model = pmo.ConcreteModel()
        self.model.n = pmo.RangeSet(1, self.x_card)
        self.model.c = pmo.RangeSet(1, self.c_card)
        self.model.A = pmo.Param(self.model.c, self.model.n, initialize=A_init)
        self.model.b = pmo.Param(self.model.c, initialize=b_init)
        self.model.Q = pmo.Param(self.model.n, self.model.n, initialize=Q_init)
        self.model.m = pmo.Param(self.model.n, initialize=m_init)
        self.model.x = pmo.Var(self.model.n)
        self.model.dual = pmo.Suffix(direction=pmo.Suffix.IMPORT)
        self.model.constraints = pmo.ConstraintList()
        for c in self.model.c:
            self.model.constraints.add(
                sum(self.model.A[c, i] * self.model.x[i] for i in self.model.n) <= self.model.b[c]
            )
        self.model.obj = pmo.Objective(
            expr=(
                0.5 * sum(sum(self.model.Q[i, j] * self.model.x[i] * self.model.x[j] for j in self.model.n) for i in self.model.n)
                + sum(self.model.m[i] * self.model.x[i] for i in self.model.n)
            )
        )
        
        # define solver
        self.solverpath = 'C:\\cygwin64\\home\\user1\\Ipopt-3.12.12\\bin\\ipopt'
        self.solver = pmo.SolverFactory('ipopt', tee=False, executable=self.solverpath)
        self.solver.options['tol'] = 1e-12
    
        # define empty output entities
        self.soln = None
        self.duals = None
    
    def solve(self):
        self.solver.solve(self.model, tee=False)
        self.soln = np.empty([self.x_card])
        for i in range(self.x_card):
            self.soln[i] = self.model.x[i+1].value
        self.duals = np.empty([self.c_card])
        for c in range(self.c_card):
            self.duals[c] = -self.model.dual[self.model.constraints[c+1]]

In [164]:
class RedundancyChecker:
    
    def __init__(self, A, b, tol=1e-9):
        self.A = A
        self.b = b
        self.tol = tol
        
        # get no of var and constraints
        self.x_card = np.shape(A)[1]
        self.c_card = np.shape(A)[0]
        
        # transform A from matrix to dict
        A_init = {}
        for i in range(self.c_card):
            for j in range(self.x_card):
                A_init[(i+1, j+1)] = A[i, j]
        
        # transform b from vector to dict
        b_init = {}
        for i in range(self.c_card):
            b_init[i+1] = b[i]
            
        # define pyomo model
        self.model = pmo.ConcreteModel()
        self.model.n = pmo.RangeSet(1, self.x_card)
        self.model.c = pmo.RangeSet(1, self.c_card)
        self.model.A = pmo.Param(self.model.c, self.model.n, initialize=A_init)
        self.model.b = pmo.Param(self.model.c, mutable=True, initialize=b_init)
        self.model.x = pmo.Var(self.model.n)
        self.model.dual = pmo.Suffix(direction=pmo.Suffix.IMPORT)
        self.model.constraints = pmo.ConstraintList()
        for c in self.model.c:
            self.model.constraints.add(
                sum(self.model.A[c, i] * self.model.x[i] for i in self.model.n) <= self.model.b[c]
            )
        
        # define solver
        self.solverpath = 'C:\\w64\\glpsol'
       # self.solver = pmo.SolverFactory('glpk', executable=self.solverpath)
#         self.solver.options['tol'] = 1e-10
        self.solver = pmo.SolverFactory('cplex')
#         self.solver.options['tol'] = 1e-10
    
        # define empty output entities
        self.redundancy = None
        self.reduced_A = None
        self.reduced_b = None
        
    def check(self):
        # for each constraint, delete any old obj, set new obj as Ax of chosen constraint
        # and maximise it.
        # Deactivate the chosen constraint itself.
        # Then check if b-Ax to see if positive (constraint is loose).
        # If so, mark as redundant.
        self.redundancy = np.zeros([self.c_card])
        self.slack = np.zeros([self.c_card])
        for c in self.model.c:
#             print('c='+ str(c))
            try:
                self.model.del_component(self.model.obj)
            except:
                pass
            self.model.b[c] += 1e-9
            self.model.obj = pmo.Objective(
                expr=-sum(self.model.A[c, i] * self.model.x[i] for i in self.model.n) +(self.model.b[c])
            )
            
            self.solver.solve(self.model, tee=False)
            self.model.b[c] -= 1e-9
            
            
            self.slack[c-1] = pmo.value(self.model.obj)
            if pmo.value(self.model.obj) > self.tol:
                self.redundancy[c-1] = 1

        self.reduced_A = self.A[self.redundancy == 0]
        self.reduced_b = self.b[self.redundancy == 0]

Problem statement

In [165]:
A = np.array(
    [[1.0, .0, -3.16515, -3.7546],
     [-1.0, .0, 3.16515, 3.7546],
     [-0.0609, .0, -0.17355, 0.2717],
     [-0.0064, .0, -0.06585, -0.4714],
     [.0, 1.0, -1.81960, 3.2841],
     [.0, -1.0, 1.81960, -3.2841],
     [.0, .0, -1.0, .0],
     [.0, .0, 1.0, .0],
     [.0, .0, .0, -1.0],
     [.0, .0, .0, 1.0], 
     [.0, .0, 3.165150, 3.754600],
     [.0, .0, 1.81960, -3.2841]
    #[.0, .0, 2.82163241,  -2.09545779],
     #  [.0, .0, 0.07350042, 0.05290033] 
    ]     
)

b = np.array(
    [0.417425, 3.582575, 0.413225, 0.467075, 1.090200, 2.909800, .0, 1.0, .0, 1.0, 3.5825751, -1.090200] #-3.582575,  #  -1.090200  # 0.043843, ,0.06334207
)

m = np.array(
    [.0, .0, .0, .0]
)

Q = np.array(
    [[0.0098*2, 0.0063, .0, .0],
     [0.0063, 0.00995*2, .0, .0],
     [.0, .0, .0, .0],
     [.0, .0, .0, .0]]
)

theta_count = 2


In [166]:
class RegionSolver:
    """
    Returns equations representing x, lambda and boundaries of a math problem in terms of theta.
    self.soln_slope
    self.soln_constant
    self.boundary_slope
    self.boundary_constant
    """
    def __init__(self, A, b, Q, m, theta_count):
        self.A = A
        self.b = b
        self.Q = Q
        self.m = m
        self.theta_count = theta_count
        self.x_count = np.shape(A)[1] - self.theta_count
        self.var_count = np.shape(A)[1]
        self.c_count = np.shape(A)[0]
        
        # returned from _solve_theta
        self.theta = None
        
        # returned from _solve_x
        self.x_problem_A = None
        self.x_problem_b = None
        self.x_problem_theta_cols = None
        self.x = None
        self.duals = None

        # returned from _get_MN
        self.M = None
        self.N = None
        self.MN = None
        
        # returned from _get_soln_params
        self.soln_slope = None
        self.soln_constant = None
        
        # returned from _set_boundaries
        self.boundary_slope = None
        self.boundary_constant = None
        
    def _solve_theta(self):
        theta_problem = GenericSolver(self.A, self.b - np.abs(self.b)*1e-3, self.Q, self.m)
        theta_problem.solve()
        self.theta = theta_problem.soln[-self.theta_count:]
        #self.theta = np.array([0, 0])
       
    def _solve_x(self):
        # define A without theta, and ignore constraints just for theta
        self.x_problem_A = self.A[:, :(self.var_count - self.theta_count)]

        # define b ignoring constraints just for theta
        self.x_problem_theta_cols = self.A[:, -self.theta_count:]
        self.x_problem_b = self.b - np.dot(self.x_problem_theta_cols, self.theta)
        
        delete_rows = []
        for r in range(self.c_count):
            if np.sum(np.abs(self.x_problem_A[r])) == 0:
                delete_rows.append(r)
        self.x_problem_A = np.delete(self.x_problem_A, delete_rows, axis=0)
        self.x_problem_b = np.delete(self.x_problem_b, delete_rows)
#         # !!!!dirty hack!!!!
#         region_problem.x_problem_b[0]=30000+0.000001
        self.x_problem_theta_cols = np.delete(self.x_problem_theta_cols, delete_rows, axis=0)
        self.x_problem_b_original = np.delete(self.b, delete_rows)
        
        # solve for x, duals
        x_problem = GenericSolver(self.x_problem_A, self.x_problem_b, self.Q, self.m)
        x_problem.solve()
        self.x = x_problem.soln
        self.duals = x_problem.duals
    
    def _get_MN(self):
        M_len = self.x_count + np.shape(self.x_problem_A)[0]
        self.M = np.zeros([M_len, M_len])
        # note it is impossible for the previous row reduction to make Q unfit for top left, 
        # because the first rows include the objective function so are impossible to be removed
        M_top_left_input = self.Q[:self.x_count, :self.x_count]
#         for i in range(M_top_left_input.shape[0]):
#             M_top_left_input[i, i] = M_top_left_input[i, i]/2.0
        self.M[:self.x_count, :self.x_count] = M_top_left_input
        self.M[:self.x_count, self.x_count:] = self.x_problem_A.T
        self.M[self.x_count:, :self.x_count] = np.multiply(self.x_problem_A.T, self.duals).T

        # if whole row is zero, multiplier is zero so delete row
        delete_rows = []
        for r in range(M_len):
            if np.sum(np.abs(self.M[r])) <= 1e-8:
                delete_rows.append(r)
        self.M = np.delete(self.M, delete_rows, axis=0)    
        self.M = np.delete(self.M, delete_rows, axis=1)
        
        # M has (no of var + no of constraints) rows.
        # For matrices theta_cols and duals, they only have rows equal to no of constraints.
        # Here we want to delete constraints that are redundant, but list delete_rows count in rows of M.
        # So count back no of var to compute rows to delete for theta_cols and duals.
        delete_rows_constraints_only = delete_rows - np.ones(len(delete_rows)) * self.x_count
        delete_rows_constraints_only = delete_rows_constraints_only.astype('int')
        
        # delete redundant rows from theta_cols, duals and N also to ensure non-singular matrix
        reduced_theta_cols = np.delete(self.x_problem_theta_cols, delete_rows_constraints_only, axis=0)
        reduced_duals = np.delete(self.duals, delete_rows_constraints_only)
        
        self.N = np.zeros([np.shape(self.M)[0], self.theta_count])
        self.N[self.x_count:] = np.multiply(reduced_theta_cols.T, reduced_duals).T
        
        MN_result = np.linalg.solve(self.M, self.N)
        self.MN = np.zeros([M_len, self.theta_count])
        kept_rows = np.delete(np.array(range(M_len)), delete_rows)
        
        for i in range(len(kept_rows)):
            self.MN[kept_rows[i], :] = MN_result[i]
        
    def _get_soln_params(self):
        self.soln_slope = -self.MN
        self.soln_constant = np.dot(-self.MN, -self.theta) + np.r_[self.x, self.duals]
        
    def _set_boundaries(self):
        # substitute x = G * theta + H into Ax <= b
        # Means AG * theta + AH <= b
        # A: x_problem_A, remove active constraints
        # b: x_problem_b, remove active constraints
        # G: soln_slope, for x (so remove lambda)
        # H: soln_constant, for x (so remove lambda)
        #
        # Then need to add back the theta theta cols into the constraints. We can use x_problem_theta_cols
        
        # formulate A, b
        sub_A = self.x_problem_A[np.abs(self.duals) <= 1e-10]        
        sub_b = self.x_problem_b_original[np.abs(self.duals) <= 1e-10]
        sub_theta_cols = self.x_problem_theta_cols[np.abs(self.duals) <= 1e-10]
        
        sub_G = self.soln_slope[:self.x_count]
        sub_H = self.soln_constant[:self.x_count]
        
        lambda_A = self.soln_slope[self.x_count:] * -1.0
        useful_lambda_A_rows = np.abs(np.sum(lambda_A, axis=1)) >= 1e-8
        lambda_A = lambda_A[useful_lambda_A_rows]
        lambda_b = self.soln_constant[self.x_count:]
        lambda_b = lambda_b[useful_lambda_A_rows]
        
        AG = np.dot(sub_A, sub_G)
        AH = np.dot(sub_A, sub_H)
        
        AG_with_theta_cols = AG + sub_theta_cols
        useful_AG_with_theta_cols = np.sum(np.abs(AG_with_theta_cols), axis=1) >= 1e-8
        AG_with_theta_cols = AG_with_theta_cols[useful_AG_with_theta_cols]
        AG_with_theta_cols = np.round(AG_with_theta_cols, decimals=6)
        
        new_rhs = sub_b - AH
        new_rhs = new_rhs[useful_AG_with_theta_cols]
        
        A_theta_only_rows = np.sum(np.abs(self.A[:, :self.x_count]), axis=1) <= 1e-8
        
        print('AG_with_theta_cols')
        print(AG_with_theta_cols)
        print('theta only rows')
        print(self.A[A_theta_only_rows][:, -self.theta_count:])
        print('lamba_A')
        print(lambda_A)
        
        boundary_slope = np.concatenate((AG_with_theta_cols,
                                         self.A[A_theta_only_rows][:, -self.theta_count:],
                                         lambda_A), axis=0)
        boundary_constant = np.concatenate((new_rhs, self.b[A_theta_only_rows], lambda_b))   
        print('boundary_slope=')
        print(boundary_slope)
        print('boundary_constant=')
        print(boundary_constant)
        reduction_problem = RedundancyChecker(boundary_slope, boundary_constant)
        reduction_problem.check()
        print('slack=')
        print(reduction_problem.slack)

        self.boundary_slope = reduction_problem.reduced_A
        self.boundary_constant = reduction_problem.reduced_b
        
    def solve(self):
        self._solve_theta()
        self._solve_x()
        self._get_MN()
        self._get_soln_params()
        self._set_boundaries()

In [167]:
region_problem = RegionSolver(A, b, Q, m, theta_count)
region_problem._solve_theta()

In [168]:
b

array([ 0.417425 ,  3.582575 ,  0.413225 ,  0.467075 ,  1.0902   ,
        2.9098   ,  0.       ,  1.       ,  0.       ,  1.       ,
        3.5825751, -1.0902   ])

In [169]:
region_problem.theta

array([0.17472503, 0.42910382])

In [170]:
# region_problem.theta = np.array([0.43, 0.6]) #array([0.6962668, 0.3672803])

In [171]:
region_problem._solve_x()

In [172]:
region_problem.x 

array([ 0.00035042, -0.00109019])

In [173]:
#region_problem.x  = np.array([ 0.0033, -0.005])

In [174]:
region_problem.duals


array([4.85971176e-14, 8.84147878e-14, 3.83637251e-13, 1.84240116e-13,
       1.94871264e-05, 3.13595023e-14])

In [175]:
region_problem.M

In [176]:
region_problem._get_MN()

In [177]:
region_problem.M

array([[1.96000000e-02, 6.30000000e-03, 0.00000000e+00],
       [6.30000000e-03, 1.99000000e-02, 1.00000000e+00],
       [0.00000000e+00, 1.94871264e-05, 0.00000000e+00]])

In [178]:
region_problem.N

array([[ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [-3.54587752e-05,  6.39976718e-05]])

In [179]:
region_problem.x_problem_theta_cols[1] * region_problem.duals[1]

array([2.79846066e-13, 3.31962162e-13])

In [180]:
region_problem._get_soln_params()

In [181]:
region_problem.soln_slope

array([[-0.58487143,  1.05560357],
       [ 1.8196    , -3.2841    ],
       [-0.        , -0.        ],
       [-0.        , -0.        ],
       [-0.        , -0.        ],
       [-0.        , -0.        ],
       [-0.03252535,  0.05870329],
       [-0.        , -0.        ]])

In [182]:
region_problem.soln_constant

array([-3.50421430e-01,  1.09020000e+00,  4.85971176e-14,  8.84147878e-14,
        3.83637251e-13,  1.84240116e-13, -1.94873251e-02,  3.13595023e-14])

In [183]:
region_problem._set_boundaries()

AG_with_theta_cols
[[-3.750021 -2.698996]
 [ 3.750021  2.698996]
 [-0.137931  0.207414]
 [-0.062107 -0.478156]]
theta only rows
[[-1.       0.     ]
 [ 1.       0.     ]
 [ 0.      -1.     ]
 [ 0.       1.     ]
 [ 3.16515  3.7546 ]
 [ 1.8196  -3.2841 ]]
lamba_A
[[ 0.03252535 -0.05870329]]
boundary_slope=
[[-3.750021   -2.698996  ]
 [ 3.750021    2.698996  ]
 [-0.137931    0.207414  ]
 [-0.062107   -0.478156  ]
 [-1.          0.        ]
 [ 1.          0.        ]
 [ 0.         -1.        ]
 [ 0.          1.        ]
 [ 3.16515     3.7546    ]
 [ 1.8196     -3.2841    ]
 [ 0.03252535 -0.05870329]]
boundary_constant=
[ 0.76784643  3.23215357  0.39188433  0.4648323   0.          1.
  0.          1.          3.5825751  -1.0902     -0.01948733]
slack=
[ 1.66381350e+00  3.38904864e-07  1.93973444e-01  6.23562449e-01
 -1.00000000e-09  5.54624999e-01  3.31963096e-01  4.58171044e-02
 -1.00000008e-09  3.54385432e-09 -6.33463976e-11]


In [184]:
region_problem.soln_slope

array([[-0.58487143,  1.05560357],
       [ 1.8196    , -3.2841    ],
       [-0.        , -0.        ],
       [-0.        , -0.        ],
       [-0.        , -0.        ],
       [-0.        , -0.        ],
       [-0.03252535,  0.05870329],
       [-0.        , -0.        ]])

In [185]:
region_problem.soln_constant

array([-3.50421430e-01,  1.09020000e+00,  4.85971176e-14,  8.84147878e-14,
        3.83637251e-13,  1.84240116e-13, -1.94873251e-02,  3.13595023e-14])

In [186]:
region_problem.boundary_slope

array([[-1.        ,  0.        ],
       [ 3.16515   ,  3.7546    ],
       [ 0.03252535, -0.05870329]])

In [187]:
region_problem.boundary_constant

array([ 0.        ,  3.5825751 , -0.01948733])

In [188]:
np.concatenate([region_problem.boundary_slope, np.array([region_problem.boundary_constant]).T], axis=1)

array([[-1.        ,  0.        ,  0.        ],
       [ 3.16515   ,  3.7546    ,  3.5825751 ],
       [ 0.03252535, -0.05870329, -0.01948733]])

In [189]:
np.array([2.821632, -2.095441])/0.043989

array([ 64.14403601, -47.63556798])

In [190]:
checker = RedundancyChecker(region_problem.boundary_slope, region_problem.boundary_constant)
checker.check()

In [191]:
region_problem.duals

array([4.85971176e-14, 8.84147878e-14, 3.83637251e-13, 1.84240116e-13,
       1.94871264e-05, 3.13595023e-14])

In [192]:
RR = np.array([[-3.750021428571, -2.698996428571],
 [ 3.750021428571,  2.698996428571],
 [-0.13793133     , 0.2074137425  ],
 [-0.062106822857 ,-0.478155862857],
 [-1.             , 0.            ],
 [ 1.             , 0.            ],
 [ 0.             ,-1.            ],
 [ 0.             , 1.            ],
 [-3.16515        ,-3.7546        ],
 [ 2.82163241206  ,-2.095457788945],
 [ 0.07350042     , 0.05290033    ],
 [ 0.03252535     ,-0.0587032875  ]])

SS = np.array([ 0.767846419362,  3.232153580638,  0.391884335561,  0.464832302916,
  0.     ,         1.      ,        0.    ,          1.,
 -3.582575 ,       0.043982036287,  0.063350211243, -0.019487324 ])

rcheck = RedundancyChecker(RR, SS)
rcheck.check()
rcheck.slack

array([ 3.34318258e+00, -1.00000008e-09,  1.84470593e-01,  7.69215852e-01,
       -1.00000000e-09,  5.54625016e-01,  5.78729125e-01, -1.00000008e-09,
       -1.00000008e-09, -9.99999895e-10,  1.06248560e-09,  9.80133454e-10])